In [2]:
import numpy as np

import sys

sys.path.append("../")

from src.futbol_types import TransitionMatrix

def build_Q(R: TransitionMatrix) -> TransitionMatrix:
    """Dada una matriz de transición de estados R,
        construye la matriz de transición de estados Q normalizando R

    Args:
        R (TransitionMatrix): Matriz de transición de estados sin normalizar

    Returns:
        Q (TransitionMatrix): Matriz de transición de estados normalizada
    """

    # Normaliza R

    # the transition probability q_g(U, V) between any two states U and V is given by

    #                                    r_g(U,V)
    # q_g(U, V) =  -----------------------------------------------------------                  (8)
    #              r_g(U,G) + r_g(U,S) + r_g(U,L) + sum_{i=1}^{11}{r_g(U,p_i)}

    Q: TransitionMatrix = np.array(R.copy())

    for i in range(14):
        if np.sum(Q[i, :]) != 0:
            # Normalize each row
            Q[i, :] = Q[i, :] / np.sum(Q[i, :])

    return Q


def psl_estimator(Q: TransitionMatrix) -> float:
    """Calcula la probabilidad de perder la pelota
        antes de tirar al arco de un lineup
        a partir de la matriz de transición de estados Q

    Args:
        Q (TransitionMatrix): Matriz de transición de estados normalizada

    Returns:
        PSL (float): Probabilidad de perder la pelota antes de tirar al arco
    """

    # Define the following block decomposition of Q_g(A)

    #           |  T_{12x12}   R_{12x2}  |
    # Q_g(A) =  |
    #           |  0_{2x12}    I_{2x2}   |

    # where T contains the transition probabilities between transient states,
    # R contains the transition probabilities from transient to absorbing states,
    # 0 is a block of all zeros and I is an identity block.
    #
    # Leveraging this decomposition, the probability of shot before loss p_g(A) can be estimated from Qq(A) as we state in the following definition.

    # p^_g(A) = [1, 0_{1×11}](I_{12x12} - T)^{-1} R[0, 1]^T

    # [1, 0_{1×11}] is a row vector of length 12 with a 1 in the first position and zeros elsewhere.
    # [0, 1]^T is a column vector of length 2 with a 1 in the second position and zeros elsewhere.

    # The matrix (I_{12x12} - T)^{-1} is the inverse of the matrix I_{12x12} - T.
    # The inverse of a matrix is a matrix that when multiplied by the original matrix gives an identity matrix.

    T: np.ndarray = Q[:12, :12]
    R: np.ndarray = Q[:12, 12:]

    M = np.eye(12) - T

    psl = 0

    try:
        M_inv = np.linalg.inv(M)
        psl = np.dot(np.dot(np.array([1] + [0] * 11), M_inv), R).dot(np.array([0, 1]).T)
    except np.linalg.LinAlgError as e:
        print("Error: La matriz es singular y no se puede invertir.")
        print(e)

    return psl

import torch
from torch import nn

class PSL(nn.Module):
    def __init__(self):
        super(PSL, self).__init__()

    def forward(self, Q):
        T = Q[:12, :12]
        R = Q[:12, 12:]

        M = torch.eye(12) - T

        psl = 0

        try:
            M_inv = torch.inverse(M)
            psl = torch.dot(torch.dot(torch.tensor([1] + [0] * 11), M_inv), R).dot(torch.tensor([0, 1]).T)
        except torch.linalg.LinAlgError as e:
            print("Error: La matriz es singular y no se puede invertir.")
            print(e)

        return psl

# Gradient Heatmap for PSL (GradCam style)

import matplotlib.pyplot as plt
import seaborn as sns

def psl_heatmap() -> None:
    """Genera un mapa de calor de la probabilidad de patear al arco antes de perder la pelota
    """



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/nacho/Desktop/code/understat-xg/.conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/nacho/Desktop/code/understat-xg/.conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/nacho/Desktop/code/understat-xg/.conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/nacho/Desktop/code/understat-xg/.conda/lib/python3.10/site-packag

In [3]:
R_storage = np.load("R_storage.npy")
R_storage.shape

FileNotFoundError: [Errno 2] No such file or directory: 'R_storage.npy'